In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Mounted at /content/drive


In [ ]:
!pip install rasterio rasterstats fiona geopandas earthpy

In [ ]:
import os
os.chdir('/content/drive/MyDrive/HEURISTICS/Ghana_predictions/Current_work')
#os.chdir('G:\My Drive\HEURISTICS\Ghana_predictions\Current_work')

In [ ]:
import geopandas as gpd
import pandas as pd
from rasterio import plot
import rasterio as rio
from rasterio.plot import show
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping
from rasterstats import zonal_stats
import folium
import fiona
from rasterio.features import shapes
import os
import glob

%matplotlib inline

In [ ]:
ls

In [ ]:

# Provide the path to your shapefile
#ashanti_cl = gpd.read_file('/content/drive/MyDrive/sepal_classification/ashanti_classification.shp')
#western_cl = gpd.read_file('/content/drive/MyDrive/sepal_classification/western_classification.shp')
#ashanti_cl = gpd.read_file(r'G:\My Drive\sepal_classification\ashanti_classification.shp')
#western_cl = gpd.read_file(r'G:\My Drive\sepal_classification\western_classification.shp')
# central_cl = gpd.read_file('/content/drive/MyDrive/sepal_classification/central_classification.shp')
# eastern_cl = gpd.read_file('/content/drive/MyDrive/sepal_classification/eastern_classification.shp')
# greater_accra_cl = gpd.read_file('/content/drive/MyDrive/sepal_classification/greater_accra_classification.shp'
#upper_east_cl = gpd.read_file('/content/drive/MyDrive/sepal_classification/upper_east_classification.shp')
#upper_west_cl = gpd.read_file('/content/drive/MyDrive/sepal_classification/upper_west_classification.shp')
#volta_cl = gpd.read_file('/content/drive/MyDrive/sepal_classification/volta_classification.shp')

In [ ]:
# central_cl = gpd.read_file('/content/drive/MyDrive/sepal_classification/central_classification.shp')
# eastern_cl = gpd.read_file('/content/drive/MyDrive/sepal_classification/eastern_classification.shp')
# greater_accra_cl = gpd.read_file('/content/drive/MyDrive/sepal_classification/greater_accra_classification.shp')


In [ ]:
# upper_east_cl = gpd.read_file('/content/drive/MyDrive/sepal_classification/upper_east_classification.shp')
# upper_west_cl = gpd.read_file('/content/drive/MyDrive/sepal_classification/upper_west_classification.shp')
# volta_cl = gpd.read_file('/content/drive/MyDrive/sepal_classification/volta_classification.shp')

In [ ]:
def process_and_save_shapefile(dataset, dataset_name, path):
    # Remove rows with 'lulc_id' equal to 0
    dataset.drop(dataset[dataset['lulc_id'] == 0].index, inplace=True)

    # Rename 'lulc_id' values and create the 'lulc_name' column
    dataset['lulc_name'] = dataset['lulc_id'].replace([1, 2, 3, 4, 5], ['forest', 'croplands', 'mining', 'settlements_bareland', 'water'])

    # Save the processed dataset to a shapefile
    dataset.to_file(f"{path}/{dataset_name}.shp")

# Example usage:
# Assuming you have a list of dataset names and corresponding GeoDataFrames
dataset_list = [
   # (ashanti_cl, 'ashanti_cl'),
    (western_cl, 'western_cl')
   # (upper_west_cl, 'upper_west_cl'),
    # (upper_east_cl, 'upper_east_cl'),
    # (greater_accra_cl, 'greater_accra_cl'),
    # (eastern_cl, 'eastern_cl'),
    # (central_cl, 'central_cl'),
    #(volta_cl, 'volta_cl')
]  # List of tuples containing (GeoDataFrame, dataset_name)
path = '/content/drive/MyDrive/sepal_classification/new_classification/'  # Your desired path

# Loop through each dataset in the list and process and save the shapefile
for dataset, dataset_name in dataset_list:
    process_and_save_shapefile(dataset, dataset_name, path)


In [ ]:
# Provide the path to your shapefile
#ashanti_cl = gpd.read_file('/content/drive/MyDrive/sepal_classification/new_classification/ashanti_cl.shp')
western_cl = gpd.read_file('/content/drive/MyDrive/sepal_classification/new_classification/western_cl.shp')


In [ ]:
western_cl.head()

,lulc_id,lulc_name,geometry
0,2,croplands,"POLYGON ((-3.11437 7.00228, -3.11410 7.00228, ..."
1,1,forest,"POLYGON ((-3.11410 7.00228, -3.11329 7.00228, ..."
2,2,croplands,"POLYGON ((-3.11329 7.00228, -3.11248 7.00228, ..."
3,2,croplands,"POLYGON ((-3.11518 7.00201, -3.11491 7.00201, ..."
4,1,forest,"POLYGON ((-3.11329 7.00201, -3.11302 7.00201, ..."


In [ ]:
# central_cl = gpd.read_file('/content/drive/MyDrive/sepal_classification/new_classification/central_cl.shp')
# eastern_cl = gpd.read_file('/content/drive/MyDrive/sepal_classification/new_classification/eastern_cl.shp')
# greater_accra_cl = gpd.read_file('/content/drive/MyDrive/sepal_classification/new_classification/greater_accra_cl.shp')
# upper_east_cl = gpd.read_file('/content/drive/MyDrive/sepal_classification/new_classification/upper_east_cl.shp')
# upper_west_cl = gpd.read_file('/content/drive/MyDrive/sepal_classification/new_classification/upper_west_cl.shp')
# volta_cl = gpd.read_file('/content/drive/MyDrive/sepal_classification/new_classification/volta_cl.shp')

In [ ]:
#upper_west_cl.head()

Plot the shapfiles

In [ ]:
# def plot_lulc_maps_with_legend(lulc_data_list, palette_list, labels_list, titles):
#     num_plots = len(lulc_data_list)

#     fig, axs = plt.subplots(1, num_plots, figsize=(6 * num_plots, 6))

#     for i in range(num_plots):
#         ax = axs[i]
#         lulc_data = lulc_data_list[i]
#         palette = palette_list[i]
#         labels = labels_list[i]
#         title = titles[i]

#         # Loop through each attribute type and plot it using the colors assigned in the palette dictionary
#         for ctype, data in lulc_data.groupby('lulc_name'):
#             # Define the color for each group using the palette dictionary
#             color = palette[ctype]

#             # Plot each group using the color defined above
#             data.plot(color=color, ax=ax, label=labels[ctype])

#         # Create a custom legend based on the palette dictionary
#         legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label=label,
#                                       markerfacecolor=color, markersize=10)
#                            for label, color in palette.items()]
#         ax.legend(handles=legend_elements, bbox_to_anchor=(0.3, 0.5), prop={'size': 12})  # Position the legend and set font size
#         ax.set(title=title)
#         ax.set_axis_off()

#     plt.show()


In [ ]:
# def plot_lulc_maps_with_legend(lulc_data_list, palette_list, labels_list, titles, output_files):
#     num_plots = len(lulc_data_list)

#     fig, axs = plt.subplots(1, num_plots, figsize=(6 * num_plots, 6))

#     for i in range(num_plots):
#         ax = axs[i]
#         lulc_data = lulc_data_list[i]
#         palette = palette_list[i]
#         labels = labels_list[i]
#         title = titles[i]
#         output_file = output_files[i]

#         # Loop through each attribute type and plot it using the colors assigned in the palette dictionary
#         for ctype, data in lulc_data.groupby('lulc_name'):
#             # Define the color for each group using the palette dictionary
#             color = palette[ctype]

#             # Plot each group using the color defined above
#             data.plot(color=color, ax=ax, label=labels[ctype])

#         # Create a custom legend based on the palette dictionary
#         legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label=label,
#                                       markerfacecolor=color, markersize=10)
#                            for label, color in palette.items()]
#         ax.legend(handles=legend_elements, bbox_to_anchor=(0.4, 0.5), prop={'size': 12})  # Position the legend and set font size
#         ax.set(title=title)
#         ax.set_axis_off()

#         # Save the plot to the specified output file
#         plt.savefig(output_file, bbox_inches='tight')

#     #plt.close(fig)  # Close the figure to free up memory



In [ ]:
# def plot_lulc_maps_with_legend(lulc_data_list, palette_list, labels_list, titles, output_files):
#     num_plots = len(lulc_data_list)

#     fig, axs = plt.subplots(1, num_plots, figsize=(6 * num_plots, 6))

#     for i in range(num_plots):
#         ax = axs[i]
#         lulc_data = lulc_data_list[i]
#         palette = palette_list[i]
#         labels = labels_list[i]
#         title = titles[i]
#         output_file = output_files[i]

#         # Loop through each attribute type and plot it using the colors assigned in the palette dictionary
#         for ctype, data in lulc_data.groupby('lulc_name'):
#             # Define the color for each group using the palette dictionary
#             color = palette[ctype]

#             # Plot each group using the color defined above
#             data.plot(color=color, ax=ax, label=labels[ctype])

#         # Create a custom legend based on the palette dictionary
#         legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label=label,
#                                       markerfacecolor=color, markersize=10)
#                            for label, color in palette.items()]
#         ax.legend(handles=legend_elements, bbox_to_anchor=(0.2, 0.5), prop={'size': 12})  # Position the legend and set font size
#         ax.set(title=title)
#         ax.set_axis_off()

#         # Save the plot to the specified output file
#         plt.savefig(output_file, bbox_inches='tight')

#     plt.close(fig)  # Close the figure to free up memory



In [ ]:
def plot_lulc_maps_with_legend(lulc_data_list, palette_list, labels_list, titles, output_files):
    for i in range(len(lulc_data_list)):
        # Create a new figure for each plot
        fig, ax = plt.subplots(figsize=(6, 6))

        lulc_data = lulc_data_list[i]
        palette = palette_list[i]
        labels = labels_list[i]
        title = titles[i]
        output_file = output_files[i]

        # Loop through each attribute type and plot it using the colors assigned in the palette dictionary
        for ctype, data in lulc_data.groupby('lulc_name'):
            # Define the color for each group using the palette dictionary
            color = palette[ctype]

            # Plot each group using the color defined above
            data.plot(color=color, ax=ax, label=labels[ctype])

        # Create a custom legend based on the palette dictionary
        legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label=label,
                                      markerfacecolor=color, markersize=10)
                           for label, color in palette.items()]
        ax.legend(handles=legend_elements, bbox_to_anchor=(0.2, 0.5), prop={'size': 12})  # Position the legend and set font size
        ax.set(title=title)
        ax.set_axis_off()

        # Save the plot to the specified output file
        plt.savefig(output_file, bbox_inches='tight')
        plt.close(fig)  # Close the figure to free up memory


In [ ]:
#ls
western_cl.head()

,lulc_id,lulc_name,geometry
0,2,croplands,"POLYGON ((-3.11437 7.00228, -3.11410 7.00228, ..."
1,1,forest,"POLYGON ((-3.11410 7.00228, -3.11329 7.00228, ..."
2,2,croplands,"POLYGON ((-3.11329 7.00228, -3.11248 7.00228, ..."
3,2,croplands,"POLYGON ((-3.11518 7.00201, -3.11491 7.00201, ..."
4,1,forest,"POLYGON ((-3.11329 7.00201, -3.11302 7.00201, ..."


In [ ]:
western_cl.lulc_name.unique()

array(['croplands', 'forest', 'settlements_bareland', 'mining'],
      dtype=object)

In [ ]:
# Create a list of LULC data
lulc_data_list = [western_cl]#,volta_cl],eastern_cl,greater_accra_cl,upper_east_cl,upper_west_cl,central_cl]

# Create a list of palette dictionaries
palette_list = [
    {'forest': 'darkgreen', 'croplands': 'limegreen', 'mining': 'red', 'settlements_bareland': 'yellow','water': 'blue' }
    # {'forest': 'darkgreen', 'croplands': 'limegreen', 'mining': 'red', 'settlements_bareland': 'yellow','water': 'blue' },
    # {'forest': 'darkgreen', 'croplands': 'limegreen', 'mining': 'red', 'settlements_bareland': 'yellow','water': 'blue' },
    # {'forest': 'darkgreen', 'croplands': 'limegreen', 'mining': 'red', 'settlements_bareland': 'yellow','water': 'blue' },
    # {'forest': 'darkgreen', 'croplands': 'limegreen', 'mining': 'red', 'settlements_bareland': 'yellow','water': 'blue' },
    #  {'forest': 'darkgreen', 'croplands': 'limegreen', 'mining': 'red', 'settlements_bareland': 'yellow','water': 'blue' }
]

# Create a list of labels dictionaries
labels_list = [
    {'forest': 'Forest', 'croplands': 'Croplands','mining': 'Mining', 'settlements_bareland': 'Settlements & Bare Lands', 'water': 'Water'}
    #    {'forest': 'Forest', 'croplands': 'Croplands','mining': 'Mining', 'settlements_bareland': 'Settlements & Bare Lands', 'water': 'Water'},
    #    {'forest': 'Forest', 'croplands': 'Croplands','mining': 'Mining', 'settlements_bareland': 'Settlements & Bare Lands', 'water': 'Water'},
    #    {'forest': 'Forest', 'croplands': 'Croplands','mining': 'Mining', 'settlements_bareland': 'Settlements & Bare Lands', 'water': 'Water'},
    #    {'forest': 'Forest', 'croplands': 'Croplands','mining': 'Mining', 'settlements_bareland': 'Settlements & Bare Lands', 'water': 'Water'},
    # {'forest': 'Forest', 'croplands': 'Croplands','mining': 'Mining', 'settlements_bareland': 'Settlements & Bare Lands', 'water': 'Water'}

]
#ashanti_cl, western_cl'lulc_ashanti.png', 'lulc_western.png','LULC Ashanti Region', 'LULC Western Region',
# Specify the output file names for each plot
output_files =['western_cl.png']
#output_files = ['volta_cl.png','eastern_cl.png','greater_accra_cl.png','upper_east_cl.png','upper_west_cl.png','central_cl.png']
# Create a list of titles
titles = ['Western Region']
#titles = ['Volta Region','Eastern Region','Greater Accra','Upper East Region','Upper West Region','Central Region']
# Plot the LULC maps with legends and save each plot
plot_lulc_maps_with_legend(lulc_data_list, palette_list, labels_list, titles, output_files)

In [ ]:
# Create a list of LULC data
lulc_data_list = [lulc_2017, lulc_2018,lulc_2019, lulc_2020]

# Create a list of palette dictionaries
palette_list = [
    {'forest': 'darkgreen', 'croplands': 'limegreen', 'settlements_bareland': 'yellow', 'mining': 'red'},
    {'forest': 'darkgreen', 'croplands': 'limegreen', 'settlements_bareland': 'yellow', 'mining': 'red'},
    {'forest': 'darkgreen', 'croplands': 'limegreen', 'settlements_bareland': 'yellow', 'mining': 'red'},
    #{1: 'darkgreen', 2: 'limegreen', 3: 'yellow', 4: 'red'},
    {'forest': 'darkgreen', 'croplands': 'limegreen', 'settlements_bareland': 'yellow', 'mining': 'red'}
]

# Create a list of labels dictionaries
labels_list = [
    {'forest': 'Forest', 'croplands': 'Croplands', 'settlements_bareland': 'Settlements & Bare Lands', 'mining': 'Mining'},
    {'forest': 'Forest', 'croplands': 'Croplands', 'settlements_bareland': 'Settlements & Bare Lands', 'mining': 'Mining'},
    {'forest': 'Forest', 'croplands': 'Croplands', 'settlements_bareland': 'Settlements & Bare Lands', 'mining': 'Mining'},
    #{1: 'Forest', 2: 'Croplands', 3: 'Settlements & Bare Lands', 4: 'Mining'},
    {'forest': 'Forest', 'croplands': 'Croplands', 'settlements_bareland': 'Settlements & Bare Lands', 'mining': 'Mining'}

]

# Specify the output file names for each plot
output_files = ['lulc_2017.png', 'lulc_2018.png', 'lulc_2019.png', 'lulc_2020.png']
# Create a list of titles
titles = ['LULC 2017', 'LULC 2018', 'LULC 2019', 'LULC 2020']
# Plot the LULC maps with legends and save each plot
plot_lulc_maps_with_legend(lulc_data_list, palette_list, labels_list, titles, output_files)


In [ ]:
# Create a list of LULC data
lulc_data_list = [lulc_2021, lulc_2022, lulc_2023]
palette_list = [
    {'forest': 'darkgreen', 'croplands': 'limegreen', 'settlements_bareland': 'yellow', 'mining': 'red'},
    {'forest': 'darkgreen', 'croplands': 'limegreen', 'settlements_bareland': 'yellow', 'mining': 'red'},
    #{'forest': 'darkgreen', 'croplands': 'limegreen', 'settlements_bareland': 'yellow', 'mining': 'red'},
    #{1: 'darkgreen', 2: 'limegreen', 3: 'yellow', 4: 'red'},
    {'forest': 'darkgreen', 'croplands': 'limegreen', 'settlements_bareland': 'yellow', 'mining': 'red'}
]

# Create a list of labels dictionaries
labels_list = [
    {'forest': 'Forest', 'croplands': 'Croplands', 'settlements_bareland': 'Settlements & Bare Lands', 'mining': 'Mining'},
    {'forest': 'Forest', 'croplands': 'Croplands', 'settlements_bareland': 'Settlements & Bare Lands', 'mining': 'Mining'},
    #{'forest': 'Forest', 'croplands': 'Croplands', 'settlements_bareland': 'Settlements & Bare Lands', 'mining': 'Mining'},
    #{1: 'Forest', 2: 'Croplands', 3: 'Settlements & Bare Lands', 4: 'Mining'},
    {'forest': 'Forest', 'croplands': 'Croplands', 'settlements_bareland': 'Settlements & Bare Lands', 'mining': 'Mining'}

]
# Create a list of titles
titles = ['LULC 2021', 'LULC 2022', 'LULC 2023']
# Specify the output file names for each plot
output_files = ['lulc_2021.png', 'lulc_2022.png', 'lulc_2023.png']

# Plot the LULC maps with legends
plot_lulc_maps_with_legend(lulc_data_list, palette_list, labels_list, titles,output_files)

Open streetmap types of landcover

In [ ]:
# Set the directory where the shapefiles are located
directory = "G:\My Drive\HEURISTICS\Ghana_predictions\Current_work\ghana-latest-free"

# Loop through each file in the directory
for filename in os.listdir(directory):
    # Check if the file is a shapefile
    if filename.endswith('.shp'):
        # Open the shapefile using Fiona
        with fiona.open(os.path.join(directory, filename)) as src:
            # Do something with the shapefile here
            print(f"Processing file {filename} with CRS: {src.crs}")

In [ ]:
waterbody_two.columns

In [ ]:
waterbody = gpd.read_file(r"G:\My Drive\HEURISTICS\Ghana_predictions\Current_work\ghana-latest-free\gis_osm_water_a_free_1.shp")
waterbody_two =gpd.read_file(r"G:\My Drive\HEURISTICS\Ghana_predictions\Current_work\ghana-latest-free\gis_osm_waterways_free_1.shp")
waterbody_two = waterbody_two[['osm_id', 'code', 'fclass', 'name', 'geometry']]
# Append the features to the combined GeoDataFrame
combined_waterbody = waterbody_two.append(waterbody, ignore_index=True)
combined_waterbody.columns = ['lulc_id', 'code', 'lulc_name', 'specific_name', 'geometry']

roads = gpd.read_file(r'G:\My Drive\HEURISTICS\Ghana_predictions\Current_work\ghana-latest-free\gis_osm_roads_free_1.shp')
roads = roads[['osm_id', 'fclass', 'name','geometry']]
roads.columns = ['lulc_id','lulc_name', 'specific_name','geometry']
landuse = gpd.read_file(r'G:\My Drive\HEURISTICS\Ghana_predictions\Current_work\ghana-latest-free\gis_osm_landuse_a_free_1.shp')
landuse.columns = ['lulc_id', 'code', 'lulc_name', 'specific_name', 'geometry']
transport = gpd.read_file(r'G:\My Drive\HEURISTICS\Ghana_predictions\Current_work\ghana-latest-free\gis_osm_transport_a_free_1.shp')
transport.columns = ['lulc_id', 'code', 'lulc_name', 'specific_name', 'geometry']
traffic = gpd.read_file(r'G:\My Drive\HEURISTICS\Ghana_predictions\Current_work\ghana-latest-free\gis_osm_traffic_free_1.shp')
traffic.columns = ['lulc_id', 'code', 'lulc_name', 'specific_name', 'geometry']

#save the shapefiles
combined_waterbody.to_file(r'G:\My Drive\HEURISTICS\Ghana_predictions\output_images\combined_waterbody.shp')
roads.to_file(r'G:\My Drive\HEURISTICS\Ghana_predictions\output_images\roads.shp')
transport.to_file(r'G:\My Drive\HEURISTICS\Ghana_predictions\output_images\transport.shp')
landuse.to_file(r'G:\My Drive\HEURISTICS\Ghana_predictions\output_images\landuse.shp')


In [ ]:
roads.columns

In [ ]:
roads = gpd.read_file(r'G:\My Drive\HEURISTICS\Ghana_predictions\Current_work\ghana-latest-free\gis_osm_roads_free_1.shp')
roads = roads[['osm_id', 'fclass', 'name','geometry']]
roads.columns = ['lulc_id','lulc_name', 'specific_name','geometry']
landuse = gpd.read_file(r'G:\My Drive\HEURISTICS\Ghana_predictions\Current_work\ghana-latest-free\gis_osm_landuse_a_free_1.shp')
landuse.columns = ['lulc_id', 'code', 'lulc_name', 'specific_name', 'geometry']
#select the land use to save
# Define a list of desired values to filter
desired_lulc = [ 'commercial', 'cemetery', 'recreation_ground','park', 'industrial', 'residential', 'military', 'retail', 'allotments']

# Filter the dataframe based on the values in the column
filtered_landuse= landuse[landuse['lulc_name'].isin(desired_lulc)]
transport = gpd.read_file(r'G:\My Drive\HEURISTICS\Ghana_predictions\Current_work\ghana-latest-free\gis_osm_transport_a_free_1.shp')
transport.columns = ['lulc_id', 'code', 'lulc_name', 'specific_name', 'geometry']
traffic = gpd.read_file(r'G:\My Drive\HEURISTICS\Ghana_predictions\Current_work\ghana-latest-free\gis_osm_traffic_free_1.shp')
traffic.columns = ['lulc_id', 'code', 'lulc_name', 'specific_name', 'geometry']

buildings = gpd.read_file(r'G:\My Drive\HEURISTICS\Ghana_predictions\Current_work\ghana-latest-free\gis_osm_buildings_a_free_1.shp')
buildings = buildings[['osm_id', 'fclass', 'name', 'geometry']]
buildings.columns =['lulc_id', 'lulc_name', 'specific_name', 'geometry']

In [ ]:
#save the shapefiles
combined_waterbody.to_file(r'G:\My Drive\HEURISTICS\Ghana_predictions\output_images\combined_waterbody.shp')
roads.to_file(r'G:\My Drive\HEURISTICS\Ghana_predictions\output_images\roads.shp')
transport.to_file(r'G:\My Drive\HEURISTICS\Ghana_predictions\output_images\transport.shp')
filtered_landuse.to_file(r'G:\My Drive\HEURISTICS\Ghana_predictions\output_images\landuse.shp')
buildings.to_file(r'G:\My Drive\HEURISTICS\Ghana_predictions\output_images\buildings.shp')